In [82]:
import os

In [83]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml\\Loan-Amount-Prediction'

In [84]:
os.chdir("../")

In [85]:
%pwd

'C:\\Users\\RICH-FILES\\Desktop\\ml'

In [86]:
project_path = "C:/Users/RICH-FILES/Desktop/ml/Loan-Amount-Prediction"

os.chdir(project_path)

In [87]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth:  int
    min_samples_split: int
    min_samples_leaf: int
    target_column: float
    

In [88]:
from credit_risk.constants import *
from credit_risk.utils.common import read_yaml, create_directories




In [89]:
#creating a Configuration class
class ConfigurationManager:
    def __init__(
        self,
        config_filepath   = CONFIG_FILE_PATH,
        params_filepath   = PARAMS_FILE_PATH,
        schema_filepath   = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)                       
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self)->ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestRegressor
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name = config.model_name,
            n_estimators=params.n_estimators,
            max_depth=params.max_depth,
            min_samples_split=params.min_samples_split,
            min_samples_leaf=params.min_samples_leaf,
            target_column=schema.name
        )
        
        return model_trainer_config
    
   
        
        
    
        

In [90]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor
import joblib

import pandas as pd

In [91]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config=config
        
    def train(self):
        train_data=pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        test_data.dropna(inplace=True)
        train_data.dropna(inplace=True)
        
        print(test_data.isnull().sum())
        print(train_data.isnull().sum())
        
        
        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop(self.config.target_column, axis=1)
        train_y=train_data[[self.config.target_column]]
        test_y=test_data[[self.config.target_column]]
        
        train_y = train_y.squeeze()
        test_y = test_y.squeeze()
        
        
        
        print("train_x", train_x.shape)
        print("train_y", train_y.shape)
        print("text_y", test_y.shape)
        print("test_x", test_x.shape)
        

        # Define features
        numerical_features = ["Income", "Emp_length", "Rate", "Percent_income"]
        categorical_features = ["Home", "Intent"]

        # Create ColumnTransformer for preprocessing
        preprocessor = ColumnTransformer(
            transformers=[
                ("num", StandardScaler(), numerical_features),
                ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output = False ), categorical_features)
            ]
        )

        # Create full pipeline with preprocessing + model
        model_pipeline = Pipeline([
            ("preprocessor", preprocessor),
            ("model",  RandomForestRegressor(n_estimators=self.config.n_estimators,
                                             max_depth=self.config.max_depth,
                                             min_samples_leaf=self.config.min_samples_leaf,
                                             min_samples_split=self.config.min_samples_split
                                             ,random_state=42))
        ])

        # Fit the pipeline (assuming X_train and y_train are defined)
        model_pipeline.fit(train_x, train_y)

        # Save the entire pipeline
        joblib.dump(model_pipeline,os.path.join(self.config.root_dir, self.config.model_name))
       
        
        #check target column
        #print(train_y[:5])
        
        #rfreg   = RandomForestRegressor(n_estimators=self.config.n_estimators, max_depth=self.config.max_depth, min_samples_leaf=self.config.min_samples_leaf, min_samples_split=self.config.min_samples_split ,random_state=42)
        
        #rfreg.fit(train_x, train_y)
       
        #joblib.dump(rfreg,os.path.join(self.config.root_dir, self.config.model_name))
        
        #Evaluation
        #'n_estimators': [100, 200, 300],
        #'max_depth': [10, 20, 30],
        #'min_samples_split': [2, 5, 10],
        #'min_samples_leaf': [1, 2, 4]
        #}
       
        #grid_search = GridSearchCV(rfreg, param_grid, cv=5, n_jobs=-1, verbose=2)
        #grid_search.fit(train_x, train_y)

        #print("Best Parameters:", grid_search.best_params_)

In [92]:
#define pipeline
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config=ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e    

[2025-03-21 23:45:46,611: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-21 23:45:46,620: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-21 23:45:46,630: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-03-21 23:45:46,634: INFO: common: created directory at: artifacts]
[2025-03-21 23:45:46,638: INFO: common: created directory at: artifacts/model_trainer]
Age               0
Income            0
Home              0
Emp_length        0
Intent            0
Amount            0
Rate              0
Percent_income    0
Cred_length       0
dtype: int64
Age               0
Income            0
Home              0
Emp_length        0
Intent            0
Amount            0
Rate              0
Percent_income    0
Cred_length       0
dtype: int64
train_x (20285, 8)
train_y (20285,)
text_y (5072,)
test_x (5072, 8)
